## Imports and reading GeoJson

In [249]:
%reset 
from keplergl import KeplerGl 
import pandas as pd
import json
import os
import geopandas as gpd
from shapely.geometry import Point

if os.path.exists('pm25_data.geojson'):
    with open('pm25_data.geojson', 'r') as f:
        geojson_data = json.load(f)

with open('pm25_config.json', 'r') as f:
    config = json.load(f)

In [251]:
# if os.path.exists('pm25_daa.geojson') == False:
geojson_data = None
file_path = 'DataLake.csv'
data = pd.read_csv(file_path, header=0)
data['TimeStampAdjustedMS'] = pd.to_datetime(data['TimeStampAdjusted']).astype(int) / 10**9
data['height'] = data['PM25']


PMData = data[['TimeStampAdjusted', 'Latitude', 'Longitude', 'PM25', 'Event','TimeStampAdjustedMS', 'height']]

def dataframe_to_geojson(df, lat_col, lon_col, properties=None):
    # Create a GeoDataFrame
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    
    # Set the coordinate reference system (CRS) to WGS84
    gdf.crs = "EPSG:4326"
    
    # Select properties to include in the GeoJSON
    if properties:
        gdf = gdf[properties + ['geometry']]
    
    # Convert to GeoJSON
    geojson = gdf.to_json()
    
    return geojson

geojson_data = dataframe_to_geojson(PMData, 'Latitude', 'Longitude', ['PM25', 'Event', 'TimeStampAdjusted', 'Latitude', 'Longitude','TimeStampAdjustedMS', 'height' ])
with open('pm25_data.geojson', 'w') as f:
    f.write(geojson_data)



In [252]:
map = KeplerGl(data={"data": geojson_data}, config = config)
map


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['data'], 'id': 'lxcvi3a4q', …

In [234]:
map.config

{'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['data'],
     'id': 'lxcvi3a4q',
     'name': ['TimeStampAdjusted'],
     'type': 'timeRange',
     'value': [1700641662000, 1707383067000],
     'enlarged': False,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None,
     'speed': 1},
    {'dataId': ['data'],
     'id': 'fk5tzhz8g',
     'name': ['Event'],
     'type': 'range',
     'value': [20, 97],
     'enlarged': False,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None,
     'speed': 1}],
   'layers': [{'id': 'c5wb7a5',
     'type': 'grid',
     'config': {'dataId': 'data',
      'label': 'pm25',
      'color': [255, 153, 31],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'Latitude', 'lng': 'Longitude'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'worldUnitSize': 0.03,
       'colorRange': {'name': 'Custom Palette',
        'type': 'custom',
        'catego